# Last FM API, but with pandas!

Spotify's API is dead to us, so we're using Last.fm's - it's still music, just not as nice of an API.

1. Create an account at https://www.last.fm/api/
2. Create an "application" to get a key: https://www.last.fm/api/account/create
    - It isn't a real application, it's just your project
    - Name/description doesn't matter, ignore callback key and callback url
3. And save the API key that shows up on the next screen

We are going to use pandas instead of "normal" Python to do this analysis.

# FIRST: SETUP

## 1) Import the libraries/packages you might need

We need a library to read in the data for us! We don't like `urllib2`, so it must be something cooler and better.

In [ ]:
# Import what you need here
import requests
import pandas as pd

## 2) Save your API key

Write your API key here so you don't forget it - it's the "api key" one, not the "shared secret" one

In [ ]:
# e8f3c22a140c30779a9a652e0e209e5a

## 3) The death of an API

I used to have some code here that allowed you to display images, but _the images don't work any more._ Let this be an important lesson: when you depend on external services, they can die at any time.

# NOW: YOUR ASSIGNMENT

## 1) Search for and print a list of 50 musicians with `lil` in their name, along with the number of listeners they have

There are a lot of musicians with "Lil" in their name - it used to be all Lil Wayne and Lil Kim, but we live in a new world now!

**I've already gotten the data for you.** Your job is to put it in pandas.

In [ ]:
# /2.0/?method=artist.search&artist=cher&api_key=YOUR_API_KEY&format=json
url = "http://ws.audioscrobbler.com/2.0/?method=artist.search&artist=lil&api_key=e8f3c22a140c30779a9a652e0e209e5a&format=json&limit=50"
response = requests.get(url)
data = response.json()

In [ ]:
df = pd.DataFrame(data['results']['artistmatches']['artist'])
df[['name', 'listeners']]

Your results should begin something like this:
    
```
Lil' Wayne has 3086628 listeners
Lily Allen has 2074266 listeners
Lil B has 194116 listeners
Lilly Wood & The Prick has 359886 listeners
Lil Ugly Mane has 31955 listeners
LIL UZI VERT has 88517 listeners
```

## 2) How many listeners does your list have in total?

The answer should be roughly **15,000,000**. If it's lower, make sure you have 50 artists instead of 30 artists.

- *Tip: What's the data type of the `listeners` count? It's going to cause a problem!*
- *Tip: If you were crazy you could use sum and a list comprehension. But you really don't have to!*

In [ ]:
df.listeners.astype(int).sum()

## 3) Show each artist's name and the URL to the extra-large image

The images don't work any more, but we'll print their URLs out anyway.

Each artist **has a list of images of different sizes**. We're interested in the second-to-last one, where `size` is `extralarge`. Print their name and use `display_image` to display their extra-large image.

- *Tip: The URL should look like this: `https://lastfm-img2.akamaized.net/i/u/300x300/0fc7d7a1812dc79e9925d80382cde594.png`*
- *Tip: You can always assume it's the second to the last, or assume it's `extralarge`, or whatever you want to do to find it.*
- *Tip: Make sure the URL is correct before you try to display it.*

Your output should look something like

```
Lil' Wayne
https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png
---
LIL UZI VERT
https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png
---
Lily Allen
https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png
---
```

(but with more people, obviously)

## 4) Find Lil Jon's `mbid` (or anyone else's!).

Oftentimes in an API, you can do a few things: you can **search** for items, and you can **see more information** about items. To find more information about the item, you need to use their **unique id**. In this dataset, it's called an `mbid` (MusicBrainz, I think - another company associated with last.fm!).

Go through the artists and print their **name and mbid**. Find Lil Jon's `mbid`. I *wanted* Lil Uzi Vert's, but for some reason it isn't there. Then I wanted us to look at Lily Allen's, but I just couldn't bring myself to do that. If you'd rather do someone else, go for it.

In [ ]:
df[df.name == "Lil Jon"]

## 5) Find the artist's name and bio using their `mbid`.

It can either be Lil Jon or whoever you selected above.

If you look at the [last.fm documentation](http://www.last.fm/api/show/artist.getInfo), you can see how to use the artist's `mbid` to find more information about them. Print **every tag associated with your artist**.

- *Tip: It's a new request to the API*
- *Tip: Use the `mbid`, and make sure you delete the `&name=Cher` from the sample endpoint*
- *Tip: If you use `print` for the bio it looks a little nicer than it would otherwise*

In [ ]:
mbid = "981d39fc-bd00-4cc6-ac67-6410f8b89098"
url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&mbid={mbid}&api_key=e8f3c22a140c30779a9a652e0e209e5a&format=json"
response = requests.get(url)
data = response.json()

In [ ]:
data['artist']['bio']['summary']

## 6) Print every tag of that artist

In [ ]:
data['artist']['tags']['tag']

In [ ]:
[tag['name'] for tag in data['artist']['tags']['tag']]

# GETTING A LITTLE CRAZY

So you know your original list of musicians? I want to get tag data for ALL OF THEM. How are we going to do that?

## 7) Find the mbid URLs

If we have a musician with an mbid of `AAA-AAA-AAA`, we get their info from a url like `http://ws.audioscrobbler.com/blahblah/?api_key=12345&mbid=AAA-AAA-AAA`.

|artist|url|
|---|---|
|`AAA-AAA-AAA`|`http://ws.audioscrobbler.com/blahblah/?api_key=12345&mbid=AAA-AAA-AAA`|
|`BBB-BBB-BBB`|`http://ws.audioscrobbler.com/blahblah/?api_key=12345&mbid=BBB-BBB-BBB`|
|`CCC-CCC-CCC`|`http://ws.audioscrobbler.com/blahblah/?api_key=12345&mbid=CCC-CCC-CCC`|

Calculate a new column called `mbid_url` for the URLs.

In [ ]:
df['mbid_url'] = "http://ws.audioscrobbler.com/blahblah/" + df.mbid
df.head()

## 7.5) Remove everyone who is missing an mbid

In [ ]:
df = df[df.mbid != ""]
df.head()

## 9) Printing our API urls

To get tag data for each artist, you need to use those `mbid` values to access their artist page on the API. Loop through the mbids, displying the URL you'll need to access.

## 10) Using the first three `mbids` and `scrape_artist` request the API urls and print the artist's info.

You built the URLs in the last question... but we aren't going to use them! It's often important to build new columns, but in this case there's a better way.

In [ ]:
def scrape_artist(row):
    # Fill in the blank for the URL mbid
    mbid = row['mbid']
    print("Requesting", mbid)

    return pd.Series({
        'mbid': mbid
    })

## 11) Using the first ten `mbids`, save the artist bio and tags as a dataframe called `df_bios`

## 12) Merge this with the original dataframe, saving it as `merged`

## 12) Only select the artists that have 'hip hop' in their tag list

## 13) What percent of "lil" results are rappers?

## 14) Seriously you are all-powerful now (it isn't cheating, it's PANDAS!)